In [68]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [69]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)

    return df3


In [70]:
def start(ticker):
  
    yahoo_financials = yfs(ticker)

    cash = yahoo_financials.get_financial_stmts('annual','cash')
    income = yahoo_financials.get_financial_stmts('annual','income')
    balance = yahoo_financials.get_financial_stmts('annual','balance')
    stock = yahoo_financials.get_key_statistics_data()[ticker]

    cash_balance = Sheet_Data(ticker,cash)
    income_balance = Sheet_Data(ticker,income)
    balance_balance = Sheet_Data(ticker,balance)
    print(ticker)
    #print(yahoo_financials.get_current_price())
    return DATA(cash_balance,income_balance,balance_balance,stock,yahoo_financials)

In [71]:
  def DATA(cash_balance,income_balance,balance_balance,stock,yahoo_financials):
    FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
    Revenue = income_balance.loc[['totalRevenue','netIncome']]
    Growth_Rate = Revenue.pct_change().mean()
    #FCF is CFO - CAPEX = FCF
    #FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

    df = pd.DataFrame()
    df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
    df['Net Income'] = FCF.loc['netIncome']
    #df['FCF/Net Income'] = df['FCF']/df['Net Income']
    minimum = (df['FCF']/df['Net Income']).min()
    return DCF_VALUE_CON(df,stock,Required_rate,Pre_rate,Cashflowgrowthrate)

In [72]:
def DCF_VALUE_CON(df,stock,Required_rate,Pre_rate,Cashflowgrowthrate):
    outstanding_shares = stock['sharesOutstanding']

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    #print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    #print(ticker)
    #print(fairvalue)
    tickers.append(ticker)
    Value.append(fairvalue)
    #print(yahoo_financials.get_current_price())

    #print(Cashflowgrowthrate)
    #print(Required_rate)
    #tickers.append(ticker)
    #ValueCon.append( fairvalue)
    #print(stock.get_current_price())
#DCF_VALUE_CON()

In [77]:
tickers = []
Value  = []
ValueCon = []
Required_rate = .15
Pre_rate = .025
Cashflowgrowthrate = .035

In [97]:
df = pd.read_csv('FINALTICKER.csv')
for i in df['Ticker'][66:]:
    ticker = i
    start(ticker)

CASS
CASY
CAT
CATM
CDAY
CDK
CDNA
CDNS
CDW
CDXS
CERN
CERS


KeyError: 'sharesOutstanding'

In [82]:
Value

[82.99576991597596,
 180.61713617426543,
 113.75325828171341,
 -17.173091101880182,
 -12.269217642641003]

In [87]:
tickers

['AAPL',
 'ABBV',
 'ABMD',
 'ACAD',
 'ACCD',
 'ACIW',
 'ACN',
 'ADBE',
 'ADI',
 'ADP',
 'ADPT',
 'ADS',
 'ADSK',
 'AGYS',
 'AKAM',
 'ALGN',
 'ALNY',
 'ALRM',
 'ALTR',
 'ALXN',
 'AMAT',
 'AMD',
 'AMGN',
 'AMRN',
 'AMSWA',
 'AMZN',
 'ANSS',
 'API',
 'APPF',
 'APPN',
 'APPS',
 'AQB',
 'ASML',
 'ATEN',
 'ATVI',
 'AVAV',
 'AVGO',
 'AVLR',
 'AVYA']

In [94]:
maindf = pd.DataFrame()
maindf['Ticker'] = tickers
maindf['Value'] = Value

In [95]:
maindf.to_csv("SAVE.csv")